#Sequence manager explanation

In [2]:
import sys
import os
sys.path.append("../..")
sys.path.append("../../kidlearn_lib/")

#Presentation

As presented in KidlearnStarter notebook, the Kidlearn library includes a set of algorithms that we develop to guide pedagogical activities. We introduce two different algorithms ZPDES that uses a simple graph of knowledge and RiARiT that includes a more complex student model. The details of the algorithms and the user studies are presented in the following paper: B. Clement, D. Roy, P.-Y. Oudeyer, M. Lopes, Multi-Armed Bandits for Intelligent Tutoring Systems, Journal of Educational Data Mining (JEDM), 2015 [(arXiv:1310.3174 [cs.AI])](http://arxiv.org/abs/1310.3174).

This algorithms are implemented in the ***seq_manager*** module of the library, with some others sequence managers used to evaluate our agorithms. In this notebook, we will describe how the module works and how define the file needed to describe an activity.

In [3]:
import kidlearn_lib as k_lib

#The sequence manager objects

There is 4 differents sequence manager wich are define for now : 
- RiARiT : an algorithms based on student skill estimation
- ZPDES : an algorithm based on the success of the student
- Sequence : a predefined sequence of activities defined by an expert
- Random : a random sequence on all activities. 

## RiARiT : The Right Activity at the Right Time

In [6]:
S = k_lib.config.seq_manager(params_file="RIARIT",directory="params_files")

##ZPDES : Zone of Proximal Development and Empirical Sucess

## Sequence : predefined by an exepert

## Random